In [1]:
import numpy as np  # probably don't need to load
import pandas as pd
#import datetime as dt
#import pandas_datareader.data as web  # probably don't need to load
#import quandl

#import blpapi
#from xbbg import blp
#import CLOutilsPyXLL as clo
import CLOComplianceUtils as ccu

#import matplotlib.pyplot as plt
import seaborn as sns
import importlib

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Read the Master Position Report

In [2]:
path = 'Z:/Shared/Risk Management and Investment Technology/CLO Optimization/CLO Reports/'
file = 'Master Position Report - BMS Aggregator as of 6.29.21.xlsx'
filepath = path + file

In [3]:
clo_df = ccu.get_master_position_report(filepath)

In [4]:
clo_df.columns

Index(['Parent Company', 'Issuer', 'Asset', 'CUSIP', 'ISIN', 'Asset Type',
       'Analyst', 'Spread', 'Floor', 'All In Rate', 'Maturity Date',
       'Mark Price', 'Adjusted Moodys Rating', 'WARF', 'MDPR', 'Moodys CFR',
       'Moodys Facility Rating', 'Moodys Issuer Outlook',
       'Moodys Issuer Watch', 'Moodys Recovery Rate', 'S&P Issuer Rating',
       'S&P Facility Rating', 'S&P Issuer Outlook', 'S&P Issuer Watch',
       'S&P Recovery', 'Agent Bank', 'CLO 4', 'CLO 5', 'CLO 6', 'CLO 7',
       'CLO 8R', 'CLO 9', 'CLO 10', 'CLO 11', 'CLO 12', 'CLO 13', 'CLO 14',
       'CLO 15', 'CLO 16', 'CLO 17', 'CLO 18', 'CLO 19', 'CLO 20', 'CLO 21',
       'Current Global Amount Outstanding', 'Moodys Industry', 'S&P Industry',
       'Lien Type', 'Issuer Country', 'Adj. WARF NEW',
       'S&P Recovery Rate (AAA)', 'S&P Global Ratings Factor', 'Libor Floor',
       'Effective Spread'],
      dtype='object')

In [10]:
clo_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 478 entries, LX190245 to LX185219
Data columns (total 54 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Parent Company                     478 non-null    object        
 1   Issuer                             478 non-null    object        
 2   Asset                              478 non-null    object        
 3   CUSIP                              453 non-null    object        
 4   ISIN                               438 non-null    object        
 5   Asset Type                         478 non-null    object        
 6   Analyst                            474 non-null    object        
 7   Spread                             472 non-null    float64       
 8   Floor                              466 non-null    float64       
 9   All In Rate                        478 non-null    float64       
 10  Maturity Date                  

# Collateral Quality Tests

In [84]:
import importlib
importlib.reload(ccu)

<module 'CLOComplianceUtils' from 'C:\\Users\\jknechtel\\Miniconda3\\RM-IT\\CLOComplianceUtils.py'>

In [125]:
pstats = ccu.Port_stats(clo_df, weight_col='CLO 20',format_output=False)
ocstats = ccu.Overcollateralization_Stats(clo_df,'CLO 20',ccu.dict_2020_20,format_output=False)
sstats = ccu.SP_CDO_Monitor_Test(clo_df,col='CLO 20')

pstats = pstats.append(ocstats)
pstats = pstats.append(sstats)
pstats

OP:  400000000.0 , NP:  397586678.61590004
Pass


,CLO 20
Min Floating Spread Test - no Libor Floors,3.669090e+00
Min Floating Spread Test - With Libor Floors,3.890123e+00
Minimum Weighted Average Coupon Test,4.005435e+00
Max Moodys Rating Factor Test (NEW WARF),2.778802e+03
Max Moodys Rating Factor Test (Orig WARF),2.929145e+03
Min Moodys Recovery Rate Test,4.773777e+01
Min S&P Recovery Rate Class A-1a,3.981928e+01
Moodys Diversity Test,8.106000e+01
Weighted Average Life Test,5.258967e+00
Percent Caa & lower,2.036717e+00


In [151]:
importlib.reload(ccu)
all_stats = ccu.all_stats_all_clos(clo_df,['CLO 4', 'CLO 5', 'CLO 6',
       'CLO 7', 'CLO 8R', 'CLO 9', 'CLO 10', 'CLO 11',
       'CLO 12', 'CLO 13', 'CLO 14', 'CLO 15',
       'CLO 16', 'CLO 17', 'CLO 18', 'CLO 19',
       'CLO 20'],ccu.dict_2020_20)

<module 'CLOComplianceUtils' from 'C:\\Users\\jknechtel\\Miniconda3\\RM-IT\\CLOComplianceUtils.py'>

C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:912: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P CLO Specified Assets'] = df[col].values
C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:540: RuntimeWarning: invalid value encountered in double_scalars
  wa = (model_df[cols[0]]*model_df[cols[1]]).sum()/model_df[cols

OP:  600000000.0 , NP:  556545498.6028999


C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:631: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P Default Rating'] = df[['S&P Facility Rating','Loan Life']].apply(lambda x: lookup_spdr(x[0],x[1]),axis=1)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:912: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P CLO Specified Assets'] = df[col].values
C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set o

Fail
OP:  400000000.0 , NP:  385234450.36660004


...

C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:615: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Loan Life']  = ((df['Maturity Date'] - pd.Timestamp.today()).dt.days/365)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:631: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P Default Rating'] = df[['S&P Facility Rating','Loan Life']].apply(lambda x: lookup_spdr(x[0],x[1]),axis=1)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:912: SettingWithCopyWarning: 
A value is try

Pass
OP:  600000000.0 , NP:  588618092.2107999


C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:631: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P Default Rating'] = df[['S&P Facility Rating','Loan Life']].apply(lambda x: lookup_spdr(x[0],x[1]),axis=1)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:912: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P CLO Specified Assets'] = df[col].values
C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set o

Pass
OP:  500000000.0 , NP:  492030239.3079001
Pass
OP:  500000000.0 , NP:  492841859.86980003
Pass
OP:  400000000.0 , NP:  397363974.5871
Pass
OP:  400000000.0 , NP:  396614583.9613
Pass
OP:  400000000.0 , NP:  397586678.6159
Pass


C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clo_df['S&P CLO Specified Assets'] = clo_df[col].values
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clo_df['Absolute Deviation'] = abs(clo_df['S&P Global Ratings Factor']-SPWARF)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:912: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [152]:
len(all_stats.index)

30

## some useful code

In [87]:
#percentage_C(clo_df,weight_col='CLO 2020-20')
clo_df.loc[(clo_df['Moodys CFR'].str.match('C'))&~clo_df['Moodys CFR'].isna(),['CLO 20']].sum()
clo_df.loc[(clo_df['MDPR'].str.match('C'))&~clo_df['MDPR'].isna(),['CLO 20']].sum()
clo_df.loc[(clo_df['Moodys Facility Rating'].str.match('C'))&~clo_df['Moodys Facility Rating'].isna(),['CLO 20']].sum()
clo_df.loc[(clo_df['Adjusted Moodys Rating'].str.match('C'))&~clo_df['Adjusted Moodys Rating'].isna(),['CLO 20']].sum()

#clo_df[['CLO 2020-20']].sum()

clo_df.loc[(clo_df['S&P Issuer Rating'].str.match('C'))&~clo_df['S&P Issuer Rating'].isna(),['CLO 20']].sum()/\
        clo_df[['CLO 20']].sum()


CLO 20    7.737976e+06
dtype: float64

CLO 20    9.670976e+06
dtype: float64

CLO 20    3776801.15
dtype: float64

CLO 20    2.748755e+07
dtype: float64

CLO 20    0.017613
dtype: float64

In [153]:
#clo_df[['Asset Type','CLO 2020-20','Spread']]
clo_df[['Asset Type','CLO 20','Spread']].info()
clo_df.loc[(clo_df['CLO 20'] > 0),['CLO 20']].info()
clo_df.loc[(clo_df['CLO 20'] == 0),['CLO 20']]

<class 'pandas.core.frame.DataFrame'>
Index: 478 entries, LX190245 to LX185219
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Asset Type  478 non-null    object 
 1   CLO 20      300 non-null    float64
 2   Spread      472 non-null    float64
dtypes: float64(2), object(1)
memory usage: 31.1+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 293 entries, LX193380 to LX185219
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   CLO 20  293 non-null    float64
dtypes: float64(1)
memory usage: 4.6+ KB


,CLO 20
LXID,
LX192319,0.0
LX192678,0.0
LX191363,0.0
LX167192,0.0
LX183733,0.0
LX179693,0.0
LX155959,0.0


In [89]:
# if no Asset Type then use LXID and str.match('US')
mask = (clo_df['Asset Type']!='Bond') & (clo_df['CLO 20'] > 0) & (~clo_df['Spread'].isna())
ccu.weighted_average(clo_df.loc[mask],['CLO 20','Spread'])
(clo_df.loc[mask,'CLO 20']*clo_df.loc[mask,'Spread']).sum()
clo_df.loc[mask,'CLO 20'].sum()


mask = (clo_df['Asset Type']=='Bond')
ccu.weighted_average(clo_df.loc[mask],['CLO 20','All In Rate'])
(clo_df.loc[mask,'CLO 20']*clo_df.loc[mask,'All In Rate']).sum()
clo_df.loc[mask,'CLO 20'].sum()


0.034290595137528156

13369402.02527225

389885389.0302

0.043111952366950895

293247.5

6802000.0

In [91]:
ccu.Excess_Weighted_Average_FloatSpread(clo_df,'CLO 20',0.032413595)

0.23886667534390452

In [92]:
#(np.floor((pd.to_datetime(df['dte']) - pd.to_datetime(dob)).dt.days / 365.25))


clo_df.loc[clo_df['Asset Type']=='Bond',['CLO 20']].sum() 
dict_2020_20 = {'Class A-1 Notes':240e6,'Class A-2 Notes':16e6,'Class B-1 Notes':39e6,'Class B-2 Notes':9e6,
               'Class C Notes':24e6,'Class D-1 Notes':16e6,'Class D-2 Notes':8e6,'Class E Notes':12e6}

CLO 20    6802000.0
dtype: float64

In [110]:
# mask = 

clo_df.loc[(clo_df['S&P Issuer Rating']!='CC') &  (clo_df['S&P Issuer Rating']!='SD') &
           (clo_df['S&P Issuer Rating']!='NR') &  (clo_df['S&P Issuer Rating']!='D'),['CLO 2020-20','S&P Issuer Rating']]

,CLO 2020-20,S&P Issuer Rating
LXID,,
LX190245,NaN,B
LX190219,NaN,B
LX175827,NaN,B-
LX193380,6.000000e+05,B-
LX170018,1.220000e+05,B-
...,...,...
LX193419,3.810000e+05,B
LX185593,2.680188e+06,B
LX192354,1.908137e+06,B


In [84]:
clo_df = add_derived_features(clo_df)
clo_df[['Adj. WARF NEW','S&P Recovery Rate (AAA)','S&P Global Ratings Factor']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 474 entries, LX190245 to LX185219
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Adj. WARF NEW              467 non-null    float64
 1   S&P Recovery Rate (AAA)    471 non-null    float64
 2   S&P Global Ratings Factor  455 non-null    float64
dtypes: float64(3)
memory usage: 14.8+ KB


In [27]:
ccu.percentage_C(clo_df, 'CLO 2020-20')*100

6.929273429715044

In [38]:
def percentage_Caa(model_df,weight_col='CLO 20'):  
    
    perC = model_df.loc[model_df['MDPR'].str.match('C'),[weight_col]].sum()/model_df[[weight_col]].sum()
    return perC.values[0]

percentage_Caa(clo_df, 'CLO 20')*100

def percentage_CCC(model_df,weight_col='CLO 20'):  
    mask = (clo_df['S&P Issuer Rating']=='CC') |  (clo_df['S&P Issuer Rating']=='C') |  \
           (clo_df['S&P Issuer Rating']=='D') | (clo_df['S&P Issuer Rating']=='SD') | \
           (clo_df['S&P Issuer Rating']=='NR') #|  (clo_df['S&P Issuer Rating']=='D')
    #perC = model_df.loc[model_df['S&P Issuer Rating'].str.match('C'),[weight_col]].sum()/model_df[[weight_col]].sum()
    perC = model_df.loc[mask,[weight_col]].sum()/model_df[[weight_col]].sum()
    return perC.values[0]

percentage_CCC(clo_df, 'CLO 20')*100

2.4379337791259466

1.7360476420831505

## CDO Test stuff


In [204]:
LL = ((clo_df['Maturity Date'] - pd.Timestamp.today()).dt.days/365)
LL
LL//1
LL%1

LXID
LX190245    7.342466
LX190219    6.345205
LX175827    4.241096
LX193380    6.794521
LX170018    3.495890
              ...   
LX193419    6.739726
LX185593    5.676712
LX192354    5.238356
LX167827    3.104110
LX185219    6.821918
Name: Maturity Date, Length: 478, dtype: float64

LXID
LX190245    7.0
LX190219    6.0
LX175827    4.0
LX193380    6.0
LX170018    3.0
           ... 
LX193419    6.0
LX185593    5.0
LX192354    5.0
LX167827    3.0
LX185219    6.0
Name: Maturity Date, Length: 478, dtype: float64

LXID
LX190245    0.342466
LX190219    0.345205
LX175827    0.241096
LX193380    0.794521
LX170018    0.495890
              ...   
LX193419    0.739726
LX185593    0.676712
LX192354    0.238356
LX167827    0.104110
LX185219    0.821918
Name: Maturity Date, Length: 478, dtype: float64

In [218]:
SPDR_Table.columns
mask = (SPDR_Table.columns == LL[0]//1) | (SPDR_Table.columns == LL[0]//1 +1)
SPDR_Table.loc['B+',mask].values[0]

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
           dtype='int64', name='Tenor')

30.011114

In [193]:
SP_CDO_Coefs('CLO 6')[1]
SP_CDO_Type('CLO 20')

4.267761

'Type II'

In [156]:
ccu.SP_CDO_Monitor_Test(clo_df,col='CLO 20')

OP:  400000000.0 , NP:  397586678.61590004
Pass


,CLO 20
S&P Weighted Average Rating Factor (SP WARF),2747.736545
Default Rate Dispersion (DRD),648.023078
Obligor Diversity Measure (ODM),192.334783
Industry Diversity Measure (IDM),16.047400
Regional Diversity Measure (RDM),1.342105
Weighted Average Life (WAL),5.237097
Break-Even Default Rate (BDR),0.674246
Adjusted Break-Even Default Rate (Adj BDR),0.688425
Scenario Default Rate (SDR),0.628258


## Dates

In [20]:
pathT = 'Z:/Shared/Risk Management and Investment Technology/CLO Optimization/'
#rein_file='End_of_Reinvestment_Period.txt'
reinvestment_end = pd.read_csv(pathT+'End_of_Reinvestment_Period.txt')
reinvestment_end['End_of_Reinvestment_Period'] = pd.to_datetime(reinvestment_end['End_of_Reinvestment_Period'])
reinvestment_end.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, CLO 4 to EUR 4
Data columns (total 1 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   End_of_Reinvestment_Period  21 non-null     datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 336.0+ bytes


In [19]:
reinvestment_end['End_of_Reinvestment_Period'] =pd.to_datetime(reinvestment_end['End_of_Reinvestment_Period'])
(reinvestment_end)

,End_of_Reinvestment_Period
CLO 4,2021-04-24
CLO 5,2023-04-26
CLO 6,2022-10-17
CLO 7,2021-07-20
CLO 8R,2024-01-17
CLO 9,2019-07-15
CLO 10,2019-10-26
CLO 11,2022-07-26
CLO 12,2020-10-18
CLO 13,2022-07-15


In [157]:
determination_dates['Fund'].unique()

array(['CLO 4', 'CLO 5', 'CLO 6', 'CLO 7', nan, 'CLO 8R', 'CLO 9',
       'CLO 10', 'CLO 11', 'CLO 12', 'CLO 13', 'CLO 14', 'CLO 15',
       'CLO 16', 'CLO 17', 'CLO 18', 'CLO 19', 'CLO 20', 'EUR 1', 'EUR 2',
       'EUR 3', 'EUR 4'], dtype=object)

In [37]:
fund_name_map = {'OCP 2013-4':'CLO 4', 'OCP 2014-5':'CLO 5', 'OCP 2014-6':'CLO 6', 'OCP 2014-7':'CLO 7',
       'OCP 2020-8R':'CLO 8R', 'OCP 2015-9':'CLO 9', 'OCP 2015-10':'CLO 10',
       'OCP 2016-11':'CLO 11', 'OCP 2016-12':'CLO 12', 'OCP 2017-13':'CLO 13', 'OCP 2017-14':'CLO 14',
       'OCP 2018-15':'CLO 15', 'OCP 2019-16':'CLO 16', 'OCP 2019-17':'CLO 17', 'OCP 2020-18':'CLO 18',
       'OCP 2020-19':'CLO 19', 'OCP 2020-20':'CLO 20', 'Euro CLO1':'EUR 1', 'Euro CLO2':'EUR 2',
       'Euro CLO3':'EUR 3', 'Euro CLO4':'EUR 4'}
determination_dates['Fund'] = determination_dates['Fund'].map(fund_name_map)

In [165]:
determination_dates

,Fund,Determination Date,Due Date,Payment Date,Determines
0,CLO 4,2021-01-14,2021-01-22,2021-01-25,14th day of each month
1,CLO 4,2021-02-12,2021-02-24,NaT,14th day of each month
2,CLO 4,2021-03-12,2021-03-24,NaT,14th day of each month
3,CLO 4,2021-04-14,2021-04-23,2021-04-26,14th day of each month
4,CLO 4,2021-05-14,2021-05-24,NaT,14th day of each month
...,...,...,...,...,...
247,EUR 4,2021-08-06,2021-08-18,NaT,Eighth Business Day after the sixth calendar day
248,EUR 4,2021-09-07,2021-09-17,NaT,Eighth Business Day after the sixth calendar day
249,EUR 4,2021-10-08,2021-10-21,2021-10-22,9th Business Day Prior
250,EUR 4,2021-11-08,2021-11-19,NaT,Eighth Business Day after the sixth calendar day


In [167]:
pmtdates = determination_dates.loc[determination_dates['Fund']=='CLO 4','Payment Date'].dropna()
pmtdates

0   2021-01-25
3   2021-04-26
6   2021-07-26
9   2021-10-22
Name: Payment Date, dtype: datetime64[ns]

In [168]:
def next_payment_date(ddates,clo_idx):
    """ Finds the closest next Determination Date for a given CLO """
    dd = ddates.loc[ddates['Fund']==clo_idx,'Payment Date'].dropna()
    next_date = min(dd.loc[dd>pd.Timestamp.today()], key=lambda s: (s-pd.Timestamp.today()))
    return next_date

next_payment_date(determination_dates,'CLO 4')

Timestamp('2021-07-26 00:00:00')

# CLO Triggers

In [104]:
pathT = 'Z:/Shared/Risk Management and Investment Technology/CLO Optimization/'
fileT = 'US CLO Triggers as of 6.24.21.xlsx'
filepathT = pathT + fileT

In [161]:
clo_list = triggers['CLOName'].unique()
#triggers[['CLOName','Title','RequirementRaw','OutcomeRaw']].groupby(['CLOName','Title']).sum()
#clo_list = clo_list.sort()
#clo_list = sorted(clo_list)

#clo_list = clo_list[11:]+clo_list[0:11]
clo_list

array(['CLO 6', 'CLO 7', 'CLO 10', 'CLO 9', 'CLO 11', 'CLO 13', 'CLO 14',
       'CLO 5', 'CLO 15', 'CLO 4', 'CLO 12', 'CLO 17', 'CLO 19', 'CLO 8R',
       'CLO 20', 'CLO 16', 'CLO 18'], dtype=object)

In [65]:
len(triggers['Title'].unique())
triggers['Title'].count()

767

1884

In [169]:
WAL_Trigger_Table = triggers.loc[triggers['Title']=='Weighted Average Life',['CLOName','PassFailStatus','Operator']]

WAL_Trigger_Table

,CLOName,PassFailStatus,Operator
108,CLO 6,In Compliance,<=
218,CLO 7,In Compliance,<=
325,CLO 10,Out of Compliance,<=
427,CLO 9,Out of Compliance,<=
539,CLO 11,In Compliance,<=
660,CLO 13,In Compliance,<=
781,CLO 14,In Compliance,<=
887,CLO 5,In Compliance,<=
1008,CLO 15,In Compliance,<=
1136,CLO 4,Out of Compliance,<=


In [67]:
# Quality Tests
#Minimum Weighted Average S&P Recovery Rate Test - Class A-1
#Minimum Floating Spread Test
#Minimum Weighted Average Coupon Test
#Moody's Diversity Test
#Maximum Moody's Rating Factor Test
#Minimum Weighted Average Moody’s Recovery Rate Test
#Weighted Average Life
cols = ['CLOName','PassFailStatus','Requirement','Outcome']

WAL_Trigger_Table = triggers.loc[triggers['Title']=='Weighted Average Life',cols]
WAS_Trigger_Table = triggers.loc[triggers['Title']=='Minimum Floating Spread Test',cols]
SPWARR_Trigger_Table = triggers.loc[triggers['Title']=='Minimum Weighted Average S&P Recovery Rate Test - Class A-1',cols]
WAC_Trigger_Table = triggers.loc[triggers['Title']=='Minimum Weighted Average Coupon Test',cols]
DIV_Trigger_Table = triggers.loc[triggers['Title']=='Moody\'s Diversity Test',cols]
WARF_Trigger_Table = triggers.loc[triggers['Title']=='Maximum Moody\'s Rating Factor Test',cols]
MWARR_Trigger_Table = triggers.loc[triggers['Title']=='Minimum Weighted Average Moody’s Recovery Rate Test',cols]

In [68]:
WAS_Trigger_Table
SPWARR_Trigger_Table
WAC_Trigger_Table
DIV_Trigger_Table
WARF_Trigger_Table
MWARR_Trigger_Table

,CLOName,PassFailStatus,Requirement,Outcome
13,CLO 6,In Compliance,0.031,0.036498
121,CLO 7,In Compliance,0.0345,0.037066
232,CLO 10,In Compliance,0.03,0.032301
339,CLO 9,Out of Compliance,0.0325,0.032341
440,CLO 11,In Compliance,0.035,0.036431
552,CLO 13,In Compliance,0.035,0.036549
672,CLO 14,In Compliance,0.0345,0.036700
795,CLO 5,In Compliance,0.032,0.037009
899,CLO 15,In Compliance,0.0345,0.036424
1024,CLO 4,In Compliance,0.0325,0.034408


,CLOName,PassFailStatus,Requirement,Outcome
6,CLO 6,In Compliance,>= 38.0%,0.3810
114,CLO 7,In Compliance,>= 40.2%,0.4030
225,CLO 10,In Compliance,>= 40.00%,0.4009
332,CLO 9,Out of Compliance,>= 40.00%,0.3985
433,CLO 11,In Compliance,>= 38.00%,0.3835
545,CLO 13,In Compliance,>= 38.0%,0.3800
666,CLO 14,In Compliance,>= 38.00%,0.3810
788,CLO 5,Out of Compliance,>= 38.00%,0.3799
893,CLO 15,In Compliance,>= 40.00%,0.4017
1143,CLO 12,In Compliance,>= 41.00%,0.4150


,CLOName,PassFailStatus,Requirement,Outcome
16,CLO 6,In Compliance,0.075,0.000000
124,CLO 7,In Compliance,0.065,0.000000
235,CLO 10,In Compliance,0.065,0.000000
342,CLO 9,In Compliance,0.065,0.000000
443,CLO 11,In Compliance,0.065,0.000000
555,CLO 13,In Compliance,0.065,0.000000
675,CLO 14,In Compliance,0.065,0.000000
798,CLO 5,In Compliance,0.065,0.000000
902,CLO 15,In Compliance,0.065,0.000000
1030,CLO 4,In Compliance,0.075,0.000000


,CLOName,PassFailStatus,Requirement,Outcome
18,CLO 6,In Compliance,>= 60,86.0
126,CLO 7,In Compliance,>= 80,86.0
237,CLO 10,Out of Compliance,>= 65,52.0
344,CLO 9,Out of Compliance,>= 60,46.0
445,CLO 11,In Compliance,>= 80,85.0
557,CLO 13,NaN,>= 80,87.0
677,CLO 14,In Compliance,>= 80,86.0
800,CLO 5,In Compliance,>= 80,85.0
904,CLO 15,In Compliance,>= 80,86.0
1033,CLO 4,Out of Compliance,>= 75,72.0


,CLOName,PassFailStatus,Requirement,Outcome
21,CLO 6,In Compliance,<= 3283,3074.0
129,CLO 7,In Compliance,<= 3236,3107.0
240,CLO 10,In Compliance,<= 3200,3052.0
347,CLO 9,Out of Compliance,<= 2883,3094.0
448,CLO 11,In Compliance,<= 3037,3025.0
560,CLO 13,In Compliance,<= 3050,3032.0
680,CLO 14,In Compliance,<= 3158,3044.0
803,CLO 5,In Compliance,<= 3377,3116.0
907,CLO 15,In Compliance,<= 3063,3001.0
1038,CLO 4,In Compliance,<= 3387,3068.0


,CLOName,PassFailStatus,Requirement,Outcome
23,CLO 6,In Compliance,>= 43.0%,0.4810
131,CLO 7,In Compliance,>= 43.00%,0.4807
243,CLO 10,In Compliance,>= 43.0%,0.4900
349,CLO 9,In Compliance,>= 46.50%,0.4885
451,CLO 11,In Compliance,>= 43.00%,0.4803
563,CLO 13,In Compliance,>= 43.00%,0.4801
683,CLO 14,In Compliance,>= 43.00%,0.4813
805,CLO 5,In Compliance,>= 43.0%,0.4820
910,CLO 15,In Compliance,>= 43.00%,0.4812
1040,CLO 4,In Compliance,>= 43.0%,0.4910


In [69]:
s = 'Minimum Weighted Average S&P Recovery Rate Test - Class A-1'
clo = 'CLO 4'

#triggers.loc[(triggers['Title']==s)&(triggers['CLOName']==clo),'Requirement'].values[0]
triggers.loc[(triggers['CLOName']==clo),['Title','Requirement']].values[0]

array(['Class B Interest Coverage Test', '>= 105.00%'], dtype=object)

In [146]:
#triggers.loc[triggers['TestType']=='Coverage Test',['CLOName','PassFailStatus','Requirement','Outcome']]
(triggers.loc[triggers['TestType']=='Coverage Test','Title'].unique())

array(['Interest Coverage Test - Class B',
       'Interest Coverage Test - Class C',
       'Interest Coverage Test - Class D',
       'Interest Coverage Test - Class A',
       'Overcollateralization Ratio Test - Class B',
       'Overcollateralization Ratio Test - Class C',
       'Overcollateralization Ratio Test - Class D',
       'Overcollateralization Ratio Test - Class A',
       'Reinvestment Overcollateralization Test', nan,
       'Overcollateralization Ratio Test - Event of Default',
       'Interest Diversion Test',
       'Overcollateralization Ratio Test - Class E',
       'Interest Coverage Test - Class A/B',
       'Overcollateralization Ratio Test - Class A/B'], dtype=object)

In [105]:
#[x.lstrip("(0123456789)/ ") for x in con_stats]
importlib.reload(ccu)
triggers = ccu.get_triggers(filepathT)
triggers

<module 'CLOComplianceUtils' from 'C:\\Users\\jknechtel\\Miniconda3\\RM-IT\\CLOComplianceUtils.py'>

,DatasetID,ReportKey,Counter,ScenarioKey,PassFailStatus,Requirement,Outcome,Title,MarketValue,TotalCap,...,CLOName,CLORank,TestName,TestResult,TestTrigger,Variance,Include,TestType,Limitation,TestRank
0,328,CMPTstGeneric50.clsTstGeneric_09_01_01,1,1,In Compliance,>= 115.00%,2.4751,Interest Coverage Test - Class B,NaN,NaN,...,CLO 6,3,Interest Coverage Test - Class B,2.4751,>= 115.00%,1.1615,NaN,Coverage Test,Interst Proceeds are used to make payments in ...,1
1,328,CMPTstGeneric50.clsTstGeneric_09_01_01_01,1,1,In Compliance,>= 110.00%,2.1547,Interest Coverage Test - Class C,NaN,NaN,...,CLO 6,3,Interest Coverage Test - Class C,2.1547,>= 110.00%,1.1110,NaN,Coverage Test,Interst Proceeds are used to make payments in ...,1
2,328,CMPTstGeneric50.clsTstGeneric_09_01_01_01_01,1,1,In Compliance,>= 105.00%,1.8116,Interest Coverage Test - Class D,NaN,NaN,...,CLO 6,3,Interest Coverage Test - Class D,1.8116,>= 105.00%,1.0605,NaN,Coverage Test,Interst Proceeds are used to make payments in ...,1
3,328,CMPTstGeneric50.clsTstGeneric_09_01_01_02,1,1,In Compliance,>= 120.00%,2.8343,Interest Coverage Test - Class A,NaN,NaN,...,CLO 6,3,Interest Coverage Test - Class A,2.8343,>= 120.00%,1.2120,NaN,Coverage Test,Interst Proceeds are used to make payments in ...,1
4,328,CMPTstGeneric7.clsTstGeneric,1,1,In Compliance,<= 5.0%,0.0000,NaN,NaN,NaN,...,CLO 6,3,(13) Moody's Counterparty Criteria - Aggregate...,0.0000,<= 5.0%,0.0495,NaN,Concentration Limitation Test,Test must be maintained or improved after givi...,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1879,1281,CMPTstGeneric7.clsTstGeneric_CL_35_02_01,3,1,In Compliance,<= 1.0%,0.0020,NaN,NaN,NaN,...,CLO 18,15,(03)(c) Single Obligor - Senior Secured Bonds ...,0.0020,<= 1.0%,0.0099,NaN,Concentration Limitation Test,Test must be maintained or improved after givi...,3
1880,1281,CMPTstGeneric7.clsTstGeneric_CL_35_02_01,4,1,In Compliance,<= 1.0%,0.0020,NaN,NaN,NaN,...,CLO 18,15,(03)(c) Single Obligor - Senior Secured Bonds ...,0.0020,<= 1.0%,0.0099,NaN,Concentration Limitation Test,Test must be maintained or improved after givi...,3
1881,1281,CMPTstGeneric7.clsTstGeneric_CL_35_02_01,5,1,In Compliance,<= 1.0%,0.0010,NaN,NaN,NaN,...,CLO 18,15,(03)(c) Single Obligor - Senior Secured Bonds ...,0.0010,<= 1.0%,0.0099,NaN,Concentration Limitation Test,Test must be maintained or improved after givi...,3
1882,1281,CMPTstGeneric7.clsTstSells,1,1,In Compliance,<= 30.0%,0.1810,NaN,NaN,NaN,...,CLO 18,15,Discretionary Sales,0.1810,<= 30.0%,0.2970,NaN,Concentration Limitation Test,Test must be maintained or improved after givi...,3


In [112]:
importlib.reload(ccu)
trigger_table_con = ccu.build_trigger_tables(ccu.trigger_df,ccu.clo_list,ccu.stats_list)
trigger_table_con

<module 'CLOComplianceUtils' from 'C:\\Users\\jknechtel\\Miniconda3\\RM-IT\\CLOComplianceUtils.py'>

Interest Coverage Test - Class A/B  for  CLO 5  DNE
Overcollateralization Ratio Test - Class A/B  for  CLO 5  DNE
Overcollateralization Ratio Test - Class E  for  CLO 5  DNE
Reinvestment Overcollateralization Test  for  CLO 5  DNE
Overcollateralization Ratio Test - Event of Default  for  CLO 5  DNE
Moody's Default Probability Rating <= Caa1 and/or S&P Rating <= CCC+  for  CLO 5  DNE
Interest Coverage Test - Class A/B  for  CLO 6  DNE
Interest Diversion Test  for  CLO 6  DNE
Overcollateralization Ratio Test - Class A/B  for  CLO 6  DNE
Overcollateralization Ratio Test - Class E  for  CLO 6  DNE
Overcollateralization Ratio Test - Event of Default  for  CLO 6  DNE
Moody's Default Probability Rating <= Caa1 and/or S&P Rating <= CCC+  for  CLO 6  DNE
Interest Coverage Test - Class A/B  for  CLO 7  DNE
Interest Coverage Test - Class D  for  CLO 7  DNE
Interest Diversion Test  for  CLO 7  DNE
Overcollateralization Ratio Test - Class A/B  for  CLO 7  DNE
Overcollateralization Ratio Test - Clas

,CLO 5,CLO 6,CLO 7,CLO 8R,CLO 10,CLO 11,CLO 12,CLO 13,CLO 14,CLO 15,CLO 16,CLO 17,CLO 18,CLO 19,CLO 20
Minimum Weighted Average S&P Recovery Rate Test - Class A-1,0.380,0.3800,0.4020,0.400,0.4000,0.380,0.4100,0.3800,0.3800,0.4000,0.4000,0.4100,0.4050,0.4000,0.3000
Minimum Floating Spread Test,0.032,0.0310,0.0345,0.020,0.0300,0.035,0.0320,0.0350,0.0345,0.0345,0.0200,0.0345,0.0200,0.0200,0.0350
Minimum Weighted Average Coupon Test,0.065,0.0750,0.0650,0.070,0.0650,0.065,0.0650,0.0650,0.0650,0.0650,0.0650,0.0700,0.0700,0.0700,0.0700
Moody's Diversity Test,80.000,60.0000,80.0000,30.000,65.0000,80.000,70.0000,80.0000,80.0000,80.0000,40.0000,75.0000,40.0000,50.0000,70.0000
Maximum Moody's Rating Factor Test,3377.000,3283.0000,3236.0000,3400.000,3200.0000,3037.000,3400.0000,3050.0000,3158.0000,3063.0000,3350.0000,2999.0000,3400.0000,3400.0000,3182.0000
Minimum Weighted Average Moody’s Recovery Rate Test,0.430,0.4300,0.4300,0.430,0.4300,0.430,0.4300,0.4300,0.4300,0.4300,NaN,0.4300,0.4300,0.4300,0.4300
Weighted Average Life,5.840,5.3200,4.7000,8.000,2.3300,5.130,3.8100,5.0700,5.4900,6.0000,7.0000,7.2500,7.2500,6.6800,8.6700
Interest Coverage Test - Class A,1.200,1.2000,1.2000,1.200,1.2000,1.200,1.2000,1.2000,1.2000,1.2000,NaN,NaN,NaN,NaN,NaN
Interest Coverage Test - Class B,1.150,1.1500,1.1500,1.100,1.1500,1.150,1.1500,1.1500,1.1500,1.1500,NaN,NaN,NaN,NaN,NaN
Interest Coverage Test - Class A/B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.2000,1.2000,1.2000,1.2000,1.2000


In [123]:
#df1 = pd.DataFrame({'A':['A0','A1','A2'],'B':['B0','B1','B2'],'C':['C0','C1','C2']},index=pd.date_range('2017-01-01',periods=3, freq='M'))
#df2 = pd.DataFrame({'A':['A3','A4','A5'],'B':['B3','B4','B5'],'C':['C3','C4','C5']},index=pd.date_range('2017-01-01',periods=3, freq='M'))

pd.concat([allstats,trigger_table_con],axis=1,keys=['Result','Trigger']).swaplevel(0,1,axis=1).sort_index(axis=1)

CLO 10  \
                                                                 Result   
Min Floating Spread Test - no Libor Floors                     3.170174   
Min Floating Spread Test - With Libor Floors                   3.398004   
Minimum Weighted Average Coupon Test                                NaN   
Max Moodys Rating Factor Test (NEW WARF)                    2827.336276   
Max Moodys Rating Factor Test (Orig WARF)                   3051.395515   
Min Moodys Recovery Rate Test                                 48.918589   
Min S&P Recovery Rate Class A-1a                              42.002528   
Moodys Diversity Test                                           52.2901   
Weighted Average Life Test                                     3.005132   
Percent Caa & lower                                            3.853893   
Percent CCC & lower                                            0.444082   
Percent 2nd Lien                                               0.287352   
End of Reinvestment Period                          2019-10-26 00:00:00   
Prior Determination Date                            2021-06-16 00:00:00   
Next Determination Date                             2021-07-16 00:00:00   
Total Portfolio Par (excl. Defaults)                     273564509.8843   
Class A/B Overcollateralization Ratio                          0.899883   
Class C Overcollateralization Ratio                            0.834038   
Class D Overcollateralization Ratio                            0.777172   
Class E Overcollateralization Ratio                            0.751551   
Reinvestment Overcollateralization Ratio                       0.751551   
S&P Weighted Average Rating Factor (SP WARF)                 2683.19824   
Default Rate Dispersion (DRD)                                923.805713   
Obligor Diversity Measure (ODM)                               84.417742   
Industry Diversity Measure (IDM)                              15.766535   
Regional Diversity Measure (RDM)                               1.295201   
Weighted Average Life (WAL)                                    3.005132   
Break-Even Default Rate (BDR)                                  0.624859   
Adjusted Break-Even Default Rate (Adj BDR)                     0.863349   
Scenario Default Rate (SDR)                                 -521.667275   
Minimum Weighted Average S&P Recovery Rate Test...                  NaN   
Minimum Floating Spread Test                                        NaN   
Moody's Diversity Test                                              NaN   
Maximum Moody's Rating Factor Test                                  NaN   
Minimum Weighted Average Moody’s Recovery Rate ...                  NaN   
Weighted Average Life                                               NaN   
Interest Coverage Test - Class A                                    NaN   
Interest Coverage Test - Class B                                    NaN   
Interest Coverage Test - Class A/B                                  NaN   
Interest Coverage Test - Class C                                    NaN   
Interest Coverage Test - Class D                                    NaN   
Interest Diversion Test                                             NaN   
Overcollateralization Ratio Test - Class A                          NaN   
Overcollateralization Ratio Test - Class B                          NaN   
Overcollateralization Ratio Test - Class A/B                        NaN   
Overcollateralization Ratio Test - Class C                          NaN   
Overcollateralization Ratio Test - Class D                          NaN   
Overcollateralization Ratio Test - Class E                          NaN   
Reinvestment Overcollateralization Test                             NaN   
Overcollateralization Ratio Test - Event of Def...                  NaN   
Cov-Lite Loans                                                      NaN   
Moody's Default Probability Rating <= Caa1 and/...                  NaN   
Moody's Rating 

In [121]:
importlib.reload(ccu)
allstats = ccu.all_stats_all_clos(clo_df, cols=ccu.clo_list, clo_dict= ccu.dict_2020_20, format_output=False)

<module 'CLOComplianceUtils' from 'C:\\Users\\jknechtel\\Miniconda3\\RM-IT\\CLOComplianceUtils.py'>

C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P CLO Specified Assets'] = df[col].values
C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:524: RuntimeWarning: invalid value encountered in double_scalars
  wa = (model_df[cols[0]]*model_df[cols[1]]).sum()/model_df[cols

OP:  400000000.0 , NP:  385234450.36660004


C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:609: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P Default Rating'] = df[['S&P Facility Rating','Loan Life']].apply(lambda x: lookup_spdr(x[0],x[1]),axis=1)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P CLO Specified Assets'] = df[col].values
C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set o

Pass
OP:  1000000000.0 , NP:  944389395.1255


...

C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:609: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P Default Rating'] = df[['S&P Facility Rating','Loan Life']].apply(lambda x: lookup_spdr(x[0],x[1]),axis=1)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P CLO Specified Assets'] = df[col].values
C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set o

Pass
OP:  600000000.0 , NP:  588618092.2107999


C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:609: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P Default Rating'] = df[['S&P Facility Rating','Loan Life']].apply(lambda x: lookup_spdr(x[0],x[1]),axis=1)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P CLO Specified Assets'] = df[col].values
C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set o

Pass
OP:  500000000.0 , NP:  492030239.3079001
Pass
OP:  500000000.0 , NP:  492841859.86980003
Pass
OP:  400000000.0 , NP:  397363974.5871
Pass
OP:  400000000.0 , NP:  396614583.9613
Pass
OP:  400000000.0 , NP:  397586678.6159
Pass


C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:621: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clo_df['S&P CLO Specified Assets'] = clo_df[col].values
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:648: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clo_df['Absolute Deviation'] = abs(clo_df['S&P Global Ratings Factor']-SPWARF)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [197]:
importlib.reload(ccu)

all_df = ccu.master_test_stats(clo_df)

<module 'CLOComplianceUtils' from 'C:\\Users\\jknechtel\\Miniconda3\\RM-IT\\CLOComplianceUtils.py'>

C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:947: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  def Effective_Spread(df,libor=0.0020):
C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:575: RuntimeWarning: invalid value encountered in double_scalars
  There seems to be two types of BDR test stats
C:\Users\jknechtel\Minico

OP:  400000000.0 , NP:  385234450.36660004


C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:666: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # uses specified assets
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:947: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  def Effective_Spread(df,libor=0.0020):
C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

Pass
OP:  1000000000.0 , NP:  944389395.1255


...

Pass
OP:  600000000.0 , NP:  588618092.2107999


C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:666: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # uses specified assets
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:947: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  def Effective_Spread(df,libor=0.0020):
C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

Pass
OP:  500000000.0 , NP:  492030239.3079001
Pass
OP:  500000000.0 , NP:  492841859.86980003
Pass


C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Obligor Diversity Measure
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:705: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  + (WAL/27.3896)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:947: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

OP:  400000000.0 , NP:  397363974.5871
Pass
OP:  400000000.0 , NP:  396614583.9613
Pass
OP:  400000000.0 , NP:  397586678.6159
Pass
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1884 entries, 0 to 1883
Data columns (total 31 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DatasetID         1884 non-null   int64         
 1   ReportKey         1884 non-null   object        
 2   Counter           1884 non-null   int64         
 3   ScenarioKey       1884 non-null   int64         
 4   PassFailStatus    1883 non-null   object        
 5   Requirement       1884 non-null   object        
 6   Outcome           1884 non-null   float64       
 7   Title             328 non-null    object        
 8   MarketValue       0 non-null      float64       
 9   TotalCap          0 non-null      float64       
 10  ScenarioName      1884 non-null   object        
 11  IsRefreshed       1884 non-null   bool          
 12  

In [198]:
all_df

CLO 10  \
                                                                 Result   
Minimum Floating Spread Test                                   0.031702   
Min Floating Spread Test - With Libor Floors                    0.03398   
Minimum Weighted Average Coupon Test                                NaN   
Max Moodys Rating Factor Test (NEW WARF)                    2827.336276   
Maximum Moody's Rating Factor Test                          3051.395515   
Minimum Weighted Average Moody’s Recovery Rate ...             0.489186   
Minimum Weighted Average S&P Recovery Rate Test...             0.420025   
Moody's Diversity Test                                          52.2901   
Weighted Average Life                                          3.005132   
Moody's Rating <= Caa1                                         0.038539   
S&P Rating <= CCC+                                             0.004441   
Percent 2nd Lien                                               0.002874   
End of Reinvestment Period                          2019-10-26 00:00:00   
Prior Determination Date                            2021-06-16 00:00:00   
Next Determination Date                             2021-07-16 00:00:00   
Next Payment Date                                   2021-07-26 00:00:00   
Total Portfolio Par (excl. Defaults)                     273564509.8843   
Overcollateralization Ratio Test - Class A/B                   0.899883   
Overcollateralization Ratio Test - Class C                     0.834038   
Overcollateralization Ratio Test - Class D                     0.777172   
Overcollateralization Ratio Test - Class E                     0.751551   
Reinvestment Overcollateralization Test                        0.751551   
S&P Weighted Average Rating Factor (SP WARF)                 2683.19824   
Default Rate Dispersion (DRD)                                923.805713   
Obligor Diversity Measure (ODM)                               84.417742   
Industry Diversity Measure (IDM)                              15.766535   
Regional Diversity Measure (RDM)                               1.295201   
Weighted Average Life (WAL)                                    3.005132   
Break-Even Default Rate (BDR)                                  0.624859   
Adjusted Break-Even Default Rate (Adj BDR)                     0.863349   
Scenario Default Rate (SDR)                                 -521.667275   
Interest Coverage Test - Class A                                    NaN   
Interest Coverage Test - Class B                                    NaN   
Interest Coverage Test - Class A/B                                  NaN   
Interest Coverage Test - Class C                                    NaN   
Interest Coverage Test - Class D                                    NaN   
Interest Diversion Test                                             NaN   
Overcollateralization Ratio Test - Class A                          NaN   
Overcollateralization Ratio Test - Class B                          NaN   
Overcollateralization Ratio Test - Event of Def...                  NaN   
Cov-Lite Loans                                                      NaN   
Moody's Default Probability Rating <= Caa1 and/...                  NaN   

                                                               \
                                                      Trigger   
Minimum Floating Spread Test                           0.0300   
Min Floating Spread Test - With Libor Floors              NaN   
Minimum Weighted Average Coupon Test                   0.0650   
Max Moodys Rating Factor Test (NEW WARF)                  NaN   
Maximum Moody's Rating Factor Test                  3200.0000   
Minimum Weighted Average Moody’s Recovery Rate ...     0.4300   
Minimum Weighted Average S&P Recovery Rate Test...     0.4000   
Moody's Diversity Test                                65.0000   
Weighted Average Life                                  2.3300   
Moody's Rating <= Caa1                           

In [178]:
#all_stats
#ccu.trigger_df.info()
#trigger_df[]
#for s in stats_list:
#    print(triggers.loc[(triggers['Title']==s)&(triggers['CLOName']=='CLO 7'),'Operator'].values[0])
trigger_direction_map = {'Minimum Floating Spread Test':'>=',
    'Minimum Weighted Average S&P Recovery Rate Test - Class A-1':'>=',
       'Minimum Floating Spread Test':'>=', 'Minimum Weighted Average Coupon Test':'>=',
       'Moody\'s Diversity Test':'>=', 'Max Moodys Rating Factor Test (NEW WARF)':'<=',
        'Maximum Moody\'s Rating Factor Test':'<=',
       'Minimum Weighted Average Moody’s Recovery Rate Test':'>=',
       'Weighted Average Life':'<=', 
        'Percent 2nd Lien':'<=',
       'End of Reinvestment Period':'=', 'Prior Determination Date':'=',
       'Next Determination Date':'=', 'Next Payment Date':'=',
       'Total Portfolio Par (excl. Defaults)':'=',                 
        'Interest Coverage Test - Class A':'>=',
       'Interest Coverage Test - Class B':'>=',
       'Interest Coverage Test - Class A/B':'>=',
       'Interest Coverage Test - Class C':'>=', 'Interest Coverage Test - Class D':'>=',
       'Interest Diversion Test':'>=', 'Overcollateralization Ratio Test - Class A':'>=',
       'Overcollateralization Ratio Test - Class B':'>=',
       'Overcollateralization Ratio Test - Class A/B':'>=',
       'Overcollateralization Ratio Test - Class C':'>=',
       'Overcollateralization Ratio Test - Class D':'>=',
       'Overcollateralization Ratio Test - Class E':'>=',
       'Reinvestment Overcollateralization Test':'>=',
       'Overcollateralization Ratio Test - Event of Default':'>=', 'Cov-Lite Loans':'<=',
       'Moody\'s Default Probability Rating <= Caa1 and/or S&P Rating <= CCC+':'<=',
       'Moody\s Rating <= Caa1':'<=', 'S&P Rating <= CCC+':'<=','Cov-Lite Loans':'<='}

trigger_direction_map

{'Minimum Floating Spread Test': '>=',
 'Minimum Weighted Average S&P Recovery Rate Test - Class A-1': '>=',
 'Minimum Weighted Average Coupon Test': '>=',
 "Moody's Diversity Test": '>=',
 'Max Moodys Rating Factor Test (NEW WARF)': '<=',
 "Maximum Moody's Rating Factor Test": '<=',
 'Minimum Weighted Average Moody’s Recovery Rate Test': '>=',
 'Weighted Average Life': '<=',
 'Percent 2nd Lien': '<=',
 'End of Reinvestment Period': '=',
 'Prior Determination Date': '=',
 'Next Determination Date': '=',
 'Next Payment Date': '=',
 'Total Portfolio Par (excl. Defaults)': '=',
 'Interest Coverage Test - Class A': '>=',
 'Interest Coverage Test - Class B': '>=',
 'Interest Coverage Test - Class A/B': '>=',
 'Interest Coverage Test - Class C': '>=',
 'Interest Coverage Test - Class D': '>=',
 'Interest Diversion Test': '>=',
 'Overcollateralization Ratio Test - Class A': '>=',
 'Overcollateralization Ratio Test - Class B': '>=',
 'Overcollateralization Ratio Test - Class A/B': '>=',
 'Over

In [139]:
all_df = ccu.all_stats_all_clos(clo_df, ccu.clo_list, ccu.dict_2020_20, format_output=False)
all_df.index = all_df.index.map(all_stats_map)


C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P CLO Specified Assets'] = df[col].values
C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:524: RuntimeWarning: invalid value encountered in double_scalars
  wa = (model_df[cols[0]]*model_df[cols[1]]).sum()/model_df[cols

OP:  400000000.0 , NP:  385234450.36660004


C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:593: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Loan Life']  = ((df['Maturity Date'] - pd.Timestamp.today()).dt.days/365)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:609: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P Default Rating'] = df[['S&P Facility Rating','Loan Life']].apply(lambda x: lookup_spdr(x[0],x[1]),axis=1)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:890: SettingWithCopyWarning: 
A value is try

Pass
OP:  1000000000.0 , NP:  944389395.1255


C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:609: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P Default Rating'] = df[['S&P Facility Rating','Loan Life']].apply(lambda x: lookup_spdr(x[0],x[1]),axis=1)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P CLO Specified Assets'] = df[col].values
C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set o

...

C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:609: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P Default Rating'] = df[['S&P Facility Rating','Loan Life']].apply(lambda x: lookup_spdr(x[0],x[1]),axis=1)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P CLO Specified Assets'] = df[col].values
C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set o

Pass
OP:  600000000.0 , NP:  588618092.2107999


C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:609: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P Default Rating'] = df[['S&P Facility Rating','Loan Life']].apply(lambda x: lookup_spdr(x[0],x[1]),axis=1)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P CLO Specified Assets'] = df[col].values
C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set o

Pass
OP:  500000000.0 , NP:  492030239.3079001
Pass
OP:  500000000.0 , NP:  492841859.86980003
Pass
OP:  400000000.0 , NP:  397363974.5871
Pass
OP:  400000000.0 , NP:  396614583.9613
Pass
OP:  400000000.0 , NP:  397586678.6159
Pass


C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P CLO Specified Assets'] = df[col].values
C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:524: RuntimeWarning: invalid value encountered in double_scalars
  wa = (model_df[cols[0]]*model_df[cols[1]]).sum()/model_df[cols

In [140]:
triggers = ccu.get_triggers(filepathT)
stats_list = ccu.qstats_list + ccu.coverage_stats + ccu.con_stats # all the triggers
trigger_table = ccu.build_trigger_tables(triggers,ccu.clo_list,stats_list)

Interest Coverage Test - Class A/B  for  CLO 5  DNE
Overcollateralization Ratio Test - Class A/B  for  CLO 5  DNE
Overcollateralization Ratio Test - Class E  for  CLO 5  DNE
Reinvestment Overcollateralization Test  for  CLO 5  DNE
Overcollateralization Ratio Test - Event of Default  for  CLO 5  DNE
Moody's Default Probability Rating <= Caa1 and/or S&P Rating <= CCC+  for  CLO 5  DNE
Interest Coverage Test - Class A/B  for  CLO 6  DNE
Interest Diversion Test  for  CLO 6  DNE
Overcollateralization Ratio Test - Class A/B  for  CLO 6  DNE
Overcollateralization Ratio Test - Class E  for  CLO 6  DNE
Overcollateralization Ratio Test - Event of Default  for  CLO 6  DNE
Moody's Default Probability Rating <= Caa1 and/or S&P Rating <= CCC+  for  CLO 6  DNE
Interest Coverage Test - Class A/B  for  CLO 7  DNE
Interest Coverage Test - Class D  for  CLO 7  DNE
Interest Diversion Test  for  CLO 7  DNE
Overcollateralization Ratio Test - Class A/B  for  CLO 7  DNE
Overcollateralization Ratio Test - Clas

In [199]:
all_df

CLO 10  \
                                                                 Result   
Minimum Floating Spread Test                                   0.031702   
Min Floating Spread Test - With Libor Floors                    0.03398   
Minimum Weighted Average Coupon Test                                NaN   
Max Moodys Rating Factor Test (NEW WARF)                    2827.336276   
Maximum Moody's Rating Factor Test                          3051.395515   
Minimum Weighted Average Moody’s Recovery Rate ...             0.489186   
Minimum Weighted Average S&P Recovery Rate Test...             0.420025   
Moody's Diversity Test                                          52.2901   
Weighted Average Life                                          3.005132   
Moody's Rating <= Caa1                                         0.038539   
S&P Rating <= CCC+                                             0.004441   
Percent 2nd Lien                                               0.002874   
End of Reinvestment Period                          2019-10-26 00:00:00   
Prior Determination Date                            2021-06-16 00:00:00   
Next Determination Date                             2021-07-16 00:00:00   
Next Payment Date                                   2021-07-26 00:00:00   
Total Portfolio Par (excl. Defaults)                     273564509.8843   
Overcollateralization Ratio Test - Class A/B                   0.899883   
Overcollateralization Ratio Test - Class C                     0.834038   
Overcollateralization Ratio Test - Class D                     0.777172   
Overcollateralization Ratio Test - Class E                     0.751551   
Reinvestment Overcollateralization Test                        0.751551   
S&P Weighted Average Rating Factor (SP WARF)                 2683.19824   
Default Rate Dispersion (DRD)                                923.805713   
Obligor Diversity Measure (ODM)                               84.417742   
Industry Diversity Measure (IDM)                              15.766535   
Regional Diversity Measure (RDM)                               1.295201   
Weighted Average Life (WAL)                                    3.005132   
Break-Even Default Rate (BDR)                                  0.624859   
Adjusted Break-Even Default Rate (Adj BDR)                     0.863349   
Scenario Default Rate (SDR)                                 -521.667275   
Interest Coverage Test - Class A                                    NaN   
Interest Coverage Test - Class B                                    NaN   
Interest Coverage Test - Class A/B                                  NaN   
Interest Coverage Test - Class C                                    NaN   
Interest Coverage Test - Class D                                    NaN   
Interest Diversion Test                                             NaN   
Overcollateralization Ratio Test - Class A                          NaN   
Overcollateralization Ratio Test - Class B                          NaN   
Overcollateralization Ratio Test - Event of Def...                  NaN   
Cov-Lite Loans                                                      NaN   
Moody's Default Probability Rating <= Caa1 and/...                  NaN   

                                                               \
                                                      Trigger   
Minimum Floating Spread Test                           0.0300   
Min Floating Spread Test - With Libor Floors              NaN   
Minimum Weighted Average Coupon Test                   0.0650   
Max Moodys Rating Factor Test (NEW WARF)                  NaN   
Maximum Moody's Rating Factor Test                  3200.0000   
Minimum Weighted Average Moody’s Recovery Rate ...     0.4300   
Minimum Weighted Average S&P Recovery Rate Test...     0.4000   
Moody's Diversity Test                                65.0000   
Weighted Average Life                                  2.3300   
Moody's Rating <= Caa1                           

In [141]:
pd.concat([all_df,trigger_table],axis=1,keys=['Result','Trigger']).swaplevel(0,1,axis=1).sort_index(axis=1)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [163]:
ccu.dict_2020_20

{'Class A-1 Notes': 240000000.0,
 'Class A-2 Notes': 16000000.0,
 'Class B-1 Notes': 39000000.0,
 'Class B-2 Notes': 9000000.0,
 'Class C Notes': 24000000.0,
 'Class D-1 Notes': 16000000.0,
 'Class D-2 Notes': 8000000.0,
 'Class E Notes': 12000000.0}

In [184]:
all_df.columns

MultiIndex([('CLO 10',  'Result'),
            ('CLO 10', 'Trigger'),
            ('CLO 11',  'Result'),
            ('CLO 11', 'Trigger'),
            ('CLO 12',  'Result'),
            ('CLO 12', 'Trigger'),
            ('CLO 13',  'Result'),
            ('CLO 13', 'Trigger'),
            ('CLO 14',  'Result'),
            ('CLO 14', 'Trigger'),
            ('CLO 15',  'Result'),
            ('CLO 15', 'Trigger'),
            ('CLO 16',  'Result'),
            ('CLO 16', 'Trigger'),
            ('CLO 17',  'Result'),
            ('CLO 17', 'Trigger'),
            ('CLO 18',  'Result'),
            ('CLO 18', 'Trigger'),
            ('CLO 19',  'Result'),
            ('CLO 19', 'Trigger'),
            ('CLO 20',  'Result'),
            ('CLO 20', 'Trigger'),
            ( 'CLO 5',  'Result'),
            ( 'CLO 5', 'Trigger'),
            ( 'CLO 6',  'Result'),
            ( 'CLO 6', 'Trigger'),
            ( 'CLO 7',  'Result'),
            ( 'CLO 7', 'Trigger'),
            ('CLO 8R

In [202]:
#all_df = all_df.stack(level=[0])
#all_df = all_df.swaplevel(0, 1)
#all_df.sort_index()
all_df.stack(level=[0]).swaplevel(0, 1).sort_index(level=[0]) #.to_csv(path+'SampleCLOComplianceStats_updated.csv')

#all_df.columns.level[1]
#all_df.pivot(index=[all_df.columns[0],all_df.index],columns=all_df.columns[1],values=all_df.values)
#all_df.groupby(by=[''])

Result  \
CLO 10 Adjusted Break-Even Default Rate (Adj BDR)               0.863349   
       Break-Even Default Rate (BDR)                            0.624859   
       Cov-Lite Loans                                                NaN   
       Default Rate Dispersion (DRD)                          923.805713   
       End of Reinvestment Period                    2019-10-26 00:00:00   
...                                                                  ...   
CLO 8R S&P Weighted Average Rating Factor (SP WARF)           2763.56546   
       Scenario Default Rate (SDR)                               0.61179   
       Total Portfolio Par (excl. Defaults)               356204755.5977   
       Weighted Average Life                                    4.872965   
       Weighted Average Life (WAL)                              4.872965   

                                                     Trigger  
CLO 10 Adjusted Break-Even Default Rate (Adj BDR)        NaN  
       Break-Even Default Rate (BDR)                     NaN  
       Cov-Lite Loans                                    0.5  
       Default Rate Dispersion (DRD)                     NaN  
       End of Reinvestment Period                        NaN  
...                                                      ...  
CLO 8R S&P Weighted Average Rating Factor (SP WARF)      NaN  
       Scenario Default Rate (SDR)                       NaN  
       Total Portfolio Par (excl. Defaults)              NaN  
       Weighted Average Life                             8.0  
       Weighted Average Life (WAL)                       NaN  

[563 rows x 2 columns]

In [205]:
#all_df
importlib.reload(ccu)
ccu.write_output(all_df,test)

<module 'CLOComplianceUtils' from 'C:\\Users\\jknechtel\\Miniconda3\\RM-IT\\CLOComplianceUtils.py'>

NameError: name 'test' is not defined

In [162]:
all_df.to_csv(path+'SampleCLOComplianceStats_updated.csv')

In [124]:
#SPDR_Table.loc['B-',1]
clo_df = ccu.SP_DR(clo_df)

# Read the Trustee files (old)

In [281]:
path = 'Z:/Shared/Risk Management and Investment Technology/CLO Optimization/'
file_prefix = 'Portfolio Assets_ Trustee Recon Detail'
suffixes = [' (CLO 2020-18)',' (CLO 2013-4)',' (CLO 2014-6)',' (CLO 2020-19)',
            ' (CLO 2020-20)',' (CLO 2014-7)',' (CLO 2015-9)',' (CLO 2020-8r)',
            ' (CLO 2016-12)',' (CLO 2017-13)',' (CLO 2017-14)',' (CLO 2018-15)',
            ' (CLO 2019-16)',' (CLO 2019-17)']

def get_trustee_recon(filepath):
    master_df = pd.read_csv(filepath,header=0)
    master_df = master_df.loc[:,~master_df.columns.str.match("Unnamed")]
    master_df.rename(columns={'LoanX Id':'LXID'},inplace=True)
    master_df.set_index('LXID', inplace=True)
    return master_df

In [3]:
# Portfolio Assets_ Trustee Recon Detail (CLO 2020-18).CSV
filepath = path+file_prefix+suffixes[13]+'.CSV'
trustee_df = get_trustee_recon(filepath)
#
#filepath = path+file_prefix+suffixes[1]+'.CSV'
#temp = get_trustee_recon(filepath)
trustee_df['Maturity Date'] = pd.to_datetime(trustee_df['Maturity Date'], infer_datetime_format=True)
trustee_df = pd.merge_asof(trustee_df.sort_values('Maturity Date'), WAL_val, left_on='Maturity Date', 
                          right_on=WAL_val.index, direction='backward', suffixes=['', '_2'])

trustee_df[['Issuer','Maturity Date','Value']]

NameError: name 'get_trustee_recon' is not defined

In [98]:
#trustee_df.merge(temp,left_on="LXID",right_on="LXID",how='left',suffixes=(suffixes[0], suffixes[1])).info()